In [46]:
extern crate rand;

use rand::Rng;

// 使用Box<dyn Fn(u32)>来表示接受单个u32参数的闭包
type OnReceivedDamage = Box<dyn Fn(u32)>;

struct Monster {
    health: u32,
    received_damage: Vec<OnReceivedDamage>,
}

// struct 的实现方法：
impl Monster {
    fn take_damage(&mut self, amount: u32) {
        let damage_received = std::cmp::min(self.health, amount);
        self.health -= damage_received;
        for callback in &mut self.received_damage {
            callback(damage_received);
        }
    }

    fn add_listener(&mut self, listener: OnReceivedDamage) {
        self.received_damage.push(listener);
    }
}

impl Default for Monster {
    fn default() -> Self {
        Monster { health: 100, received_damage: Vec::new() }
    }
}

#[derive(Default)]
struct DamageCounter {
    damage_inflicted: u32,
}

impl DamageCounter {
    fn reached_target_damage(&self) -> bool {
        self.damage_inflicted > 100
    }

    fn on_damage_received(&mut self, damage: u32) {
        self.damage_inflicted += damage;
    }
}

use rand;

fn main() {
    let mut rng = rand::thread_rng();
    let mut counter = DamageCounter::default();
    let mut monsters: Vec<_> = (0..5).map(|_| Monster::default()).collect();

    for monster in &mut monsters {
        monster.add_listener(Box::new(|damage| counter.on_damage_received(damage)));
    }

    while !counter.reached_target_damage() {
        let index = rng.gen_range(0..monsters.len());
        let target = &mut monsters[index];

        let damage = rng.gen_range(0..50);
        target.take_damage(damage);

        println!("Monster {} received {} damage", index, damage);
    }
}

main();


// 作者：Rust_Magazine
// 链接：https://juejin.cn/post/7042571788349341703


Error: can't find crate for `rand`

In [93]:
use std::path::Path;
use std::fs::File;

use std::error::Error;
use std::io::{self, BufRead};

struct Dictionary {
    words: Vec<String>,
}

fn read_lines<P>(filename: P) -> io::Result<io::Lines<io::BufReader<File>>>
    where P: AsRef<Path>, {
        let file = File::open(filename)?;
        Ok(io::BufReader::new(file).lines())
}

impl Dictionary {
    // The output is wrapped in a Result to allow matching on errors
    // Returns an Iterator to the Reader of the lines of the file.    
  fn from_file(filename: & String) -> Result<Self, Box<dyn Error>> {
      
    let file_path = std::path::Path::new(filename);
    let mut words = Vec::new();
      
    if let Ok(lines) = read_lines(file_path) {
        // Consumes the iterator, returns an (Optional) String
        for line in lines {
            println!("line: {}", line.as_ref().expect("msg"));
            if let Ok(ip) = line {
                println!("{}", ip);
                words.push(String::from(ip));
                
            }
        }
    }
      
    Ok(Dictionary { words })
  }
}

fn main() -> Result<(), Box<dyn Error>> {
    println!("loading dictionary");
    let dict_path = String::from("/etc/issue");
    let dict = Dictionary::from_file(&dict_path)?;
    println!("dict:{}", dict.words.len());
    Ok(())
}

main();

loading dictionary
filename: /etc/issue
line: Debian GNU/Linux 11 \n \l
Debian GNU/Linux 11 \n \l
line: 

dict:2
